In [24]:
from collections import defaultdict
import queue
import heapq
import os
import sys

f = open(os.path.join(sys.path[0], "input.txt"), "r")
lines = f.readlines()
f.close()
data = []
for line in lines:
  tmp = []
  for x in line.strip().split():
    tmp.append(int(x))
  data.append(tmp)

data = [sorted(x) for x in data]

class myQueue:
  def __init__(self):
    self.arr = []
  def size(self):
    return len(self.arr)
  def push(self, item):
    return heapq.heappush(self.arr, item)
  def pop(self):
    return heapq.heappop(self.arr)
  def has(self, item):
    return item in self.arr
  def get_priority(self, val):
    for x in self.arr:
      if x[1] == val:
        return x[0]
    return None
  def replace(self, val, priority):
    for i in range(len(self.arr)):
      if self.arr[i][1] == val:
        self.arr[i][0] = priority
        heapify(self.arr)
        return True
    return False
    


class Maze:
  def __init__(self, data):
    self.edge = data[0][0]
    self.size = self.edge**2
    self.start = 0
    self.goal = data[-1][0]
    self.graph = data[1:-1]

  def get_path(self, node, path):
    res = [node]
    while path[node] != -1:
      node = path[node]
      res.append(node)
    return res[::-1]

  def Heuristic(self, node):
    rows = abs((self.goal % self.edge) - (node % self.edge))
    cols = abs(int(self.goal / self.edge) - int(node / self.edge))
    return rows + cols

  def BFS(self):
    frontier = [self.start]
    visited = []
    path = {self.start: -1}

    while len(frontier) > 0:
      cur = frontier.pop(0)
      if not cur in visited:
        visited.append(cur)
        successors = self.graph[cur]
        for x in successors:
          if x not in visited and x not in frontier:
            path[x] = cur
            if x == self.goal:
              visited.append(x)
              return True, len(visited), visited, self.get_path(x, path)
            frontier.append(x)
    return False, len(visited), visited, self.get_path(cur, path)
  
  def UCS(self):
    explore = []
    path = {self.start: -1}
    frontier = myQueue()
    frontier.push((0, self.start))
    while frontier.size() > 0:
      cur_cost, cur_node = frontier.pop()
      explore.append(cur_node)
      if cur_node == self.goal:
        return True, len(explore), explore, self.get_path(cur_node, path)
      successors = self.graph[cur_node]
      for x in successors:
        x_not_in_frontier = frontier.get_priority(x) is None
        x_cost = cur_cost + 1
        if not x in explore and x_not_in_frontier:
          frontier.push((x_cost, x))
          path[x] = cur_node
        elif not x_not_in_frontier and x_cost < frontier.get_priority(x):
          frontier.replace(x, x_cost)
    return False, len(explore), explore, self.get_path(cur_node, path)
  
  def DLS(self, start, visited, path, depth):
    copy_path = path[:]

    if depth == 0 or start in path:
      return False, visited, copy_path
    
    visited.append(start)
    copy_path.append(start)

    if start == self.goal:
      return True, visited, copy_path

    successors = self.graph[start]
    for x in successors:
      res = self.DLS(x, visited, copy_path, depth - 1)
      if res[0]:
        return res
    return False, visited, copy_path
  
  def IDS(self):
    found = False
    visited = []
    path = []
    times = 0
    for i in range(self.size):
      found, v, path = self.DLS(self.start, [], [], i+1)
      visited.append(v)
      times += len(v)
      if found:
        return True, times, visited, path
    return False, times, visited, path

  def GBFS(self):
    visited = []
    path = {self.start: -1}
    frontier = myQueue()
    frontier.push((self.Heuristic(self.start), self.start))
    while frontier.size() > 0:
      cur_cost, cur_node = frontier.pop()
      visited.append(cur_node)
      if cur_node == self.goal:
        return True, len(visited), visited, self.get_path(cur_node, path)
      successors = self.graph[cur_node]
      for x in successors:
        x_not_in_frontier = frontier.get_priority(x) is None
        x_cost = self.Heuristic(x)
        if not x in visited and x_not_in_frontier:
          frontier.push((x_cost, x))
          path[x] = cur_node
        elif not x_not_in_frontier and x_cost < frontier.get_priority(x):
          frontier.replace(x, x_cost)
    return False, len(visited), visited, self.get_path(cur_node, path)

  def A_star(self):
    visited = []
    path = {self.start: -1}
    frontier = myQueue()
    frontier.push((self.Heuristic(self.start), self.start))
    while frontier.size() > 0:
      cur_cost, cur_node = frontier.pop()
      visited.append(cur_node)
      if cur_node == self.goal:
        return True, len(visited), visited, self.get_path(cur_node, path)
      successors = self.graph[cur_node]
      for x in successors:
        x_cost = self.Heuristic(x) + len(self.get_path(cur_node, path))
        if x != path[cur_node]:
          frontier.push((x_cost, x))
          path[x] = cur_node
    return False, len(visited), visited, self.get_path(cur_node, path)


maze = Maze(data)

f = open(os.path.join(sys.path[0], "output.txt"), "w")
searchs_name = ["1. Breadth-first search", "2. Uniform-cost search", "3. Iterative deepening search", "4. Greedy-best first search", "5. Tree-search A*"]
results = [maze.BFS(), maze.UCS(), maze.IDS(), maze.GBFS(), maze.A_star()]

for i in range(len(searchs_name)):
  name = searchs_name[i] + " (Start node: {}; Goal node: {})\nResult:\n".format(maze.start, maze.goal)
  f.write(name)
  print (name[:-2])
  if results[i][0]:
    time = " - The time to escape the maze: {} minutes\n".format(results[i][1])
    if type(results[i][2][0]) == int:
      explored = " - The list of explored nodes: {}\n".format(results[i][2])
    else:
      explored = " - The list of explored nodes:\n"
      for j in range(len(results[i][2])):
        explored += "    * Depth = {}: {}\n".format(j, results[i][2][j])
    path = " - The list of nodes on the path found: {}\n\n".format(results[i][3])
    f.write(" - Found the path to goal\n")
    print (" - Found the path to goal")
    f.write(time)
    print (time[:-2])
    f.write(explored)
    print (explored[:-1])
    f.write(path)
    print (path)
  else:
    f.write(" - The path to goal not found\n\n")
    print (" - The path to goal not found\n\n")

f.close()

print (">>> The result is also printed to file output.txt in the same folder <<<")
